In [1]:
from lxml import etree
from pathlib import Path
from openpyxl import Workbook
import collections

In [2]:
def get_flattened_show(node,data):
    for item in node:
        if len(item)==0:
            text= item.text.strip() if item.text  else ''
            if item.tag in data.keys():
                if isinstance(data[item.tag],list):
                    data[item.tag].append(text)
                else:
                    data[item.tag]=[data[item.tag]]
            else:
                data[item.tag] = text
        else:
            # print(item.tag,end="--")
            get_flattened_show(item,data)

def split_list_attributes(shows):
    for show in shows:
        key_to_change=[]
        for key in show.keys():
            if isinstance(show[key],list):
                key_to_change.append(key)
        for key in key_to_change:
            key_values=show.pop(key)
#         print(key)
            if key=="Fullname":
                show[key]=key_values[0]
            else:
                for index,value in enumerate(key_values):
                    show[f"{key}_{index}"]=value

                    
def equalize_attributes(shows):
    keys=set()
    for show in shows:
        for key in show.keys():
            keys.add(key)
    for show in shows:
        for key in keys:
            if not key in show.keys():
                show[key]=''
                    

def sorte_shows(shows):
    return [ collections.OrderedDict(sorted(show.items())) for show in shows]
                    
                    
                    
def get_shows(shows):
    list_shows=[]
    for show in shows:
        showDict={}
        data={}
        get_flattened_show(show,data)
        list_shows.append(data)
    split_list_attributes(list_shows)
    equalize_attributes(list_shows)
    return list_shows

In [3]:
path = Path("./Archivdaten_2019")
wb=Workbook(write_only=True)
parser = etree.XMLParser(recover=True)

In [4]:
files=[file for file in path.iterdir() if not file.is_dir()]

In [5]:
for file in files:
    file=file.absolute()
    print(f"working on {file.name}")
    tree = etree.parse(str(file),parser=parser)
    shows=tree.xpath(".//daten/einfuegen/sendung//stammdaten")
    new_shows=get_shows(shows)
    sorted_shows=sorte_shows(new_shows)
    ws=wb.create_sheet(file.stem)
    ws.append(list(sorted_shows[0].keys()))
    for data in sorted_shows:
        try:
            ws.append(list(data.values()))
        except MemoryError as e:
            print(e)

working on 2019_3SAT.xml
working on 2019_ARD.xml
working on 2019_ARTE.xml
working on 2019_BFS.xml
working on 2019_DMAX.xml
working on 2019_DW-tv.xml
working on 2019_EinsFestival.xml
working on 2019_HF.xml
working on 2019_K1.xml
working on 2019_K1DOKU.xml
working on 2019_KKA.xml
working on 2019_MDR.xml
working on 2019_N24DOKU.xml
working on 2019_N3_Hamburg.xml
working on 2019_N3_Radio_Bremen.xml
working on 2019_NTV.xml
working on 2019_PHOENIX.xml
working on 2019_PRO7.xml
working on 2019_ProSiebenFun.xml
working on 2019_ProSieben_MAXX.xml
working on 2019_rbb_Berlin.xml
working on 2019_RTL.xml
working on 2019_RTL2.xml
working on 2019_RTLplus.xml
working on 2019_RTLS.xml
working on 2019_RTL_Nitro.xml
working on 2019_SAT1.xml
working on 2019_Sat1Gold.xml
working on 2019_sixx.xml
working on 2019_Sport1.xml
working on 2019_SWR_BW.xml
working on 2019_SWR_SR.xml
working on 2019_tagesschau24.xml
working on 2019_Tele_5.xml
working on 2019_VOX.xml
working on 2019_WDR.xml
working on 2019_WELT.xml
w

In [6]:
wb.save("my.xlsx")